# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset (consider using a left join)
combined_data = pd.merge(school_data, student_data, how = 'left')
combined_data.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


## District Summary

In [2]:
# Calculate the Totals (Schools and Students)
total_schools = combined_data["School ID"].nunique()
total_students = combined_data["Student ID"].nunique()
# Calculate the Total Budget
total_budget = combined_data["budget"].unique().sum()
# Calculate the Average Scores
average_reading = combined_data["reading_score"].mean()
average_math = combined_data["math_score"].mean()
# Calculate the Percentage Pass Rates
reading_pass = combined_data.loc[combined_data["reading_score"] >= 70]
per_reading_pass = (len(reading_pass) / total_students)*100
math_pass = combined_data.loc[combined_data["math_score"] >= 70]
per_math_pass = (len(math_pass) / total_students)*100
both_pass = combined_data.loc[reading_pass.index & math_pass.index]
per_both_pass = (len(both_pass)/total_students)*100
# Minor Data Cleanup
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                       "Total Students": [total_students],
                       "Total Budget": [total_budget],
                       "Average Reading Score": [average_reading],
                        "Average Math Score": [average_math],
                        "% Passing Reading": [per_reading_pass],
                       "% Passing Math": [per_math_pass],
                       "% Passing Both": [per_both_pass]})
district_summary["Total Budget"] = district_summary["Total Budget"].map("${0:,.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{0:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{0:,.2f}".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{0:,.3f}%".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{0:,.3f}%".format)
district_summary["% Passing Both"] = district_summary["% Passing Both"].map("{0:,.3f}%".format)

# Display the data frame
district_summary

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Both
0,15,39170,"$24,649,428.00",81.88,78.99,85.805%,74.981%,65.172%


## School Summary

In [3]:
school = combined_data.groupby(['school_name'])
# Determine the School Type
school_type = school['type'].unique()
# Calculate the total student count
student_count = school['student_name'].count()
# Calculate the total school budget and per capita spending
school_budget = school['budget'].mean()
per_capita = school_budget/student_count
# Calculate the average test scores
school_average_reading = school['reading_score'].mean()
school_average_math = school['math_score'].mean()
# Calculate the passing scores by creating a filtered data frame
school_pass_reading =combined_data[combined_data["reading_score"]>=70]
school_per_pass_reading = school_pass_reading.groupby("school_name").count()["reading_score"]/student_count * 100
school_pass_math =combined_data[combined_data["math_score"]>=70]
school_per_pass_math = school_pass_math.groupby("school_name").count()["math_score"]/student_count * 100
school_average_per_pass = (school_per_pass_reading + school_per_pass_math)/2
# Convert to data frame
school_summary = pd.DataFrame({"School Type": (school_type),
                                        "Total Students": (student_count),
                                        "Total School Budget": (school_budget),
                                        "Per Capita Budget": (per_capita),
                                        "Average Reading Score": (school_average_reading),
                                        "Average Math Score": (school_average_math),
                                        "% Passing Reading": (school_per_pass_reading),
                                        "% Passing Math": (school_per_pass_math),
                                        "Average % Passing": (school_average_per_pass)})

# Minor data munging
del school_summary.index.name
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${0:,.2f}".format)
school_summary["Per Capita Budget"] = school_summary["Per Capita Budget"].map("${0:,.2f}".format)
school_summary["Average Reading Score"] = school_summary["Average Reading Score"].map("{0:,.2f}".format)
school_summary["Average Math Score"] = school_summary["Average Math Score"].map("{0:,.2f}".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{0:,.3f}%".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{0:,.3f}%".format)
school_summary["Average % Passing"] = school_summary["Average % Passing"].map("{0:,.3f}%".format)

# Display the data frame
school_summary

,School Type,Total Students,Total School Budget,Per Capita Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Average % Passing
Bailey High School,[District],4976,"$3,124,928.00",$628.00,81.03,77.05,81.933%,66.680%,74.307%
Cabrera High School,[Charter],1858,"$1,081,356.00",$582.00,83.98,83.06,97.040%,94.133%,95.587%
Figueroa High School,[District],2949,"$1,884,411.00",$639.00,81.16,76.71,80.739%,65.988%,73.364%
Ford High School,[District],2739,"$1,763,916.00",$644.00,80.75,77.10,79.299%,68.310%,73.804%
Griffin High School,[Charter],1468,"$917,500.00",$625.00,83.82,83.35,97.139%,93.392%,95.266%
Hernandez High School,[District],4635,"$3,022,020.00",$652.00,80.93,77.29,80.863%,66.753%,73.808%
Holden High School,[Charter],427,"$248,087.00",$581.00,83.81,83.80,96.253%,92.506%,94.379%
Huang High School,[District],2917,"$1,910,635.00",$655.00,81.18,76.63,81.316%,65.684%,73.500%
Johnson High School,[District],4761,"$3,094,650.00",$650.00,80.97,77.07,81.222%,66.058%,73.640%
Pena High School,[Charter],962,"$585,858.00",$609.00,84.04,83.84,95.946%,94.595%,95.270%


## Top Performing Schools (By Passing Rate)

In [4]:
# Sort and show top five schools
top_schools = school_summary.sort_values(['Average % Passing'], ascending = False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Capita Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Average % Passing
Cabrera High School,[Charter],1858,"$1,081,356.00",$582.00,83.98,83.06,97.040%,94.133%,95.587%
Thomas High School,[Charter],1635,"$1,043,130.00",$638.00,83.85,83.42,97.309%,93.272%,95.291%
Pena High School,[Charter],962,"$585,858.00",$609.00,84.04,83.84,95.946%,94.595%,95.270%
Griffin High School,[Charter],1468,"$917,500.00",$625.00,83.82,83.35,97.139%,93.392%,95.266%
Wilson High School,[Charter],2283,"$1,319,574.00",$578.00,83.99,83.27,96.540%,93.868%,95.204%


## Bottom Performing Schools (By Passing Rate)

In [5]:
# Sort and show bottom five schools
bottom_schools = school_summary.sort_values(['Average % Passing'], ascending = True)
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Capita Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Average % Passing
Rodriguez High School,[District],3999,"$2,547,363.00",$637.00,80.74,76.84,80.220%,66.367%,73.293%
Figueroa High School,[District],2949,"$1,884,411.00",$639.00,81.16,76.71,80.739%,65.988%,73.364%
Huang High School,[District],2917,"$1,910,635.00",$655.00,81.18,76.63,81.316%,65.684%,73.500%
Johnson High School,[District],4761,"$3,094,650.00",$650.00,80.97,77.07,81.222%,66.058%,73.640%
Ford High School,[District],2739,"$1,763,916.00",$644.00,80.75,77.10,79.299%,68.310%,73.804%


## Math Scores by Grade

In [6]:
# Create data series of scores by grade levels using conditionals
school_reduced_m = combined_data.loc[:, ['school_name', 'grade', 'math_score']]
ninth_graders_m = school_reduced_m.loc[school_reduced_m['grade'] == "9th"]
tenth_graders_m = school_reduced_m.loc[school_reduced_m['grade'] == "10th"]
eleventh_graders_m = school_reduced_m.loc[school_reduced_m['grade'] == "11th"]
twelth_graders_m = school_reduced_m.loc[school_reduced_m['grade'] == "12th"]
# Group each by school name
ninth_average_m = ninth_graders_m.groupby('school_name')['math_score'].mean()
tenth_average_m = tenth_graders_m.groupby('school_name')['math_score'].mean()
eleventh_average_m = eleventh_graders_m.groupby('school_name')['math_score'].mean()
twelth_average_m = twelth_graders_m.groupby('school_name')['math_score'].mean()
# Combine series into single data frame
math_by_grade = pd.DataFrame({"9th": (ninth_average_m),
                              "10th": (tenth_average_m),
                              "11th": (eleventh_average_m),
                              "12th": (twelth_average_m)})

# Minor data munging
del math_by_grade.index.name
# Display the data frame
math_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [7]:
# Create data series of scores by grade levels using conditionals
school_reduced_r = combined_data.loc[:, ['school_name', 'grade', 'reading_score']]
ninth_graders_r = school_reduced_r.loc[school_reduced_r['grade'] == "9th"]
tenth_graders_r = school_reduced_r.loc[school_reduced_r['grade'] == "10th"]
eleventh_graders_r = school_reduced_r.loc[school_reduced_r['grade'] == "11th"]
twelth_graders_r = school_reduced_r.loc[school_reduced_r['grade'] == "12th"]
# Group each by school name
ninth_average_r = ninth_graders_r.groupby('school_name')['reading_score'].mean()
tenth_average_r = tenth_graders_r.groupby('school_name')['reading_score'].mean()
eleventh_average_r = eleventh_graders_r.groupby('school_name')['reading_score'].mean()
twelth_average_r = twelth_graders_r.groupby('school_name')['reading_score'].mean()
# Combine series into single data frame
reading_by_grade = pd.DataFrame({"9th": (ninth_average_r),
                              "10th": (tenth_average_r),
                              "11th": (eleventh_average_r),
                              "12th": (twelth_average_r)})
# Minor data munging
del reading_by_grade.index.name
# Display the data frame
reading_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [8]:
#copied from earlier cell
spending = combined_data.groupby(['school_name'])
student_count = spending['student_name'].count()
school_budget = spending['budget'].mean()
per_capita = school_budget/student_count
school_average_reading = spending['reading_score'].mean()
school_average_math = spending['math_score'].mean()
school_pass_reading =combined_data[combined_data["reading_score"]>=70]
school_per_pass_reading = school_pass_reading.groupby("school_name").count()["reading_score"]/student_count * 100
school_pass_math =combined_data[combined_data["math_score"]>=70]
school_per_pass_math = school_pass_math.groupby("school_name").count()["math_score"]/student_count * 100
school_average_per_pass = (school_per_pass_reading + school_per_pass_math)/2
spending_summary = pd.DataFrame({
                                "Average Reading Score": (school_average_reading),
                                "Average Math Score": (school_average_math),
                                "% Passing Reading": (school_per_pass_reading),
                                "% Passing Math": (school_per_pass_math),
                                "Average % Passing": (school_average_per_pass)})
# Establish the bins -- choose any set of bins you would like, but see below for testing bins
# to test, set your bins as follows: [0, 585, 615, 645, 675]
# ALSO -- Note that the values for `% Passing Math`, `% Passing Reading` and `% Overall Passing Rate`
# were computed using averages of averages -- your results may vary if you use weighted averages
bins = [0, 585, 615, 645, 675]
# Categorize the spending based on the bins
values = ["Low (<$585)", "Fair ($585-615)", "Moderate($615-645)", "High ($645-675)"]
# Assemble into data frame
spending_summary["Per Capita Spending"] = pd.cut(per_capita, bins, labels=values, include_lowest = True, right = False)
spending_summary = spending_summary.groupby("Per Capita Spending")
# Minor data munging

# Display results
spending_summary.mean()

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Average % Passing
Per Capita Spending,,,,,
Low (<$585),83.933814,83.455399,96.610877,93.460096,95.035486
Fair ($585-615),83.885211,83.599686,95.900287,94.230858,95.065572
Moderate($615-645),81.891436,79.079225,86.106569,75.668212,80.887391
High ($645-675),81.027843,76.997210,81.133951,66.164813,73.649382


## Scores by School Size

In [9]:
#copied from earlier cell
size = combined_data.groupby(['school_name'])
student_count = size['student_name'].count()
school_budget = size['budget'].mean()
per_capita = school_budget/student_count
school_average_reading = size['reading_score'].mean()
school_average_math = size['math_score'].mean()
school_pass_reading =combined_data[combined_data["reading_score"]>=70]
school_per_pass_reading = school_pass_reading.groupby("school_name").count()["reading_score"]/student_count * 100
school_pass_math =combined_data[combined_data["math_score"]>=70]
school_per_pass_math = school_pass_math.groupby("school_name").count()["math_score"]/student_count * 100
school_average_per_pass = (school_per_pass_reading + school_per_pass_math)/2

size_summary = pd.DataFrame({
                                "Average Reading Score": (school_average_reading),
                                "Average Math Score": (school_average_math),
                                "% Passing Reading": (school_per_pass_reading),
                                "% Passing Math": (school_per_pass_math),
                                "Average % Passing": (school_average_per_pass)})
# Establish the bins 
bins = [0, 1500, 3000, 5000]
values = ["Small (<1500)", "Medium (1500-3000)", "Large (>3000)"]

# Categorize the size based on the bins
size_summary["School Size"] = pd.cut(student_count, bins, labels=values, include_lowest = True, right = False)
size_summary = size_summary.groupby("School Size")

# Display results
size_summary.mean()

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Average % Passing
School Size,,,,,
Small (<1500),83.892148,83.664898,96.445946,93.497607,94.971776
Medium (1500-3000),82.822740,80.904987,90.588593,83.556977,87.072785
Large (>3000),80.919864,77.063340,81.059691,66.464293,73.761992


## Scores by School Type

In [14]:
# Type | Average Math Score | Average Reading Score | % Passing Math | % Passing Reading | % Overall Passing Rate
schooltype = combined_data.groupby(['type'])
student_count = schooltype['student_name'].count()
type_average_reading = schooltype['reading_score'].mean()
type_average_math = schooltype['math_score'].mean()
type_pass_reading =combined_data[combined_data["reading_score"]>=70]
type_per_pass_reading = type_pass_reading.groupby("type").count()["reading_score"]/student_count * 100
type_pass_math =combined_data[combined_data["math_score"]>=70]
type_per_pass_math = type_pass_math.groupby("type").count()["math_score"]/student_count * 100
type_average_per_pass = (type_per_pass_reading + type_per_pass_math)/2
# Assemble into data frame
type_summary = pd.DataFrame({"Average Reading Score": (type_average_reading),
                                "Average Math Score": (type_average_math),
                                "% Passing Reading": (type_per_pass_reading),
                                "% Passing Math": (type_per_pass_math),
                                "Average % Passing": (type_average_per_pass)})
# Minor data munging
del type_summary.index.name
# Display results
type_summary

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Average % Passing
Charter,83.902821,83.406183,96.645891,93.701821,95.173856
District,80.962485,76.987026,80.905249,66.518387,73.711818
